# Staffing and overtime trends for firefighters in California

Staffing shortages are forcing many local fire departments in rural areas of Northern California to turn down service requests for larger fires and overwork their staff. This notebook reshapes and exports wages data from the California State Controller's Office. Data visualizations produced with the data are available at https://observablehq.com/@carolineghisolfi/california-firefighters. 

In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

In [2]:
ff_payroll = pd.read_csv('../data/processed/ff_payroll.csv', index_col=0)
ff_payroll.reset_index(drop=True, inplace=True)
ff_payroll.head()

,year,employertype,employername,departmentorsubdivision,position,overtimepay,employercounty,adjusted_overtime
0,2010,County,Alameda,Fire District - Alaco,FIREFIGHTER,0.0,Alameda,0.0
1,2010,County,Alameda,Fire District - Alaco,FIREFIGHTER,0.0,Alameda,0.0
2,2010,County,Alameda,Fire District - Alaco,FIREFIGHTER,0.0,Alameda,0.0
3,2010,County,Alameda,Fire District - Alaco,FIREFIGHTER,0.0,Alameda,0.0
4,2010,County,Alameda,Fire District - Alaco,FIREFIGHTER,0.0,Alameda,0.0


In [8]:
ff_overtime_byyear = ff_payroll[ff_payroll.year > 2010].groupby(['year', 'employertype'])\
    .agg(avg_adjusted_overtime = ('adjusted_overtime', 'mean')).reset_index()

overtime_dfs = []

for tp in ff_overtime_byyear.employertype.unique():
    tp_df = ff_overtime_byyear[ff_overtime_byyear.employertype == tp]
    tp_df['pct_change'] = (tp_df.avg_adjusted_overtime.pct_change() * 100).round(1)
    tp_df['pct_change'] = np.where(tp_df['pct_change'] > 0, 
                                   '+' + tp_df['pct_change'].astype(str) + '%',
                                   tp_df['pct_change'].astype(str) + '%'
                                  )
    tp_df['pct_change'] = tp_df['pct_change'].str.replace('nan%', '')
    overtime_dfs.append(tp_df)

ff_overtime_byyear = pd.concat(overtime_dfs)

ff_overtime_byyear.head()

,year,employertype,avg_adjusted_overtime,pct_change
0,2011,City,21780.322793,
4,2012,City,25208.374666,+15.7%
8,2013,City,27539.108328,+9.2%
12,2014,City,28868.563061,+4.8%
16,2015,City,31101.910306,+7.7%


In [9]:
ff_1920_counts = ff_payroll[ff_payroll.year >= 2019]\
    .groupby(['year', 'employertype']).position.count().reset_index()\
    .pivot(index='employertype', columns='year', values='position').reset_index()\
    .add_prefix('ff_')

ff_1920_counts['diff'] = ff_1920_counts.ff_2020 - ff_1920_counts.ff_2019
ff_1920_counts['change_desc'] = np.where(ff_1920_counts['diff'] > 0, 'increase', 'decrease')
ff_1920_counts['pct_change'] = (ff_1920_counts['diff'] / ff_1920_counts.ff_2019 * 100).round(1)
ff_1920_counts['pct_change'] = np.where(ff_1920_counts['pct_change'] > 0, 
                               '+' + ff_1920_counts['pct_change'].astype(str) + '%',
                               ff_1920_counts['pct_change'].astype(str) + '%')

ff_1920_counts.sort_values('diff', inplace=True)

ff_1920_counts

year,ff_employertype,ff_2019,ff_2020,diff,change_desc,pct_change
0,City,10172,9675,-497,decrease,-4.9%
2,Special District,4280,3873,-407,decrease,-9.5%
1,County,3883,3777,-106,decrease,-2.7%
3,State Department,3324,3769,445,increase,+13.4%


In [6]:
ff_overtime_byyear.to_csv('../data/processed/ff_overtime_byyear.csv')
ff_1920_counts.to_csv('../data/processed/ff_1920_counts.csv')